In [1]:
import sys
sys.path.append("../../")
from data.Frame import Frame
from bk import show_weighted_average
from bokeh.io import output_notebook
from bokeh.plotting import figure, output_file, show, reset_output
reset_output()
output_notebook()

Loading BokehJS ...

In [2]:
years = list(map(str, list(range(2010,2018))))

In [3]:
show_weighted_average(years, 'NV.AGR.TOTL.ZS', 'NY.GDP.MKTP.CD', ['LDCs', 'Developing excluding LDCs'])

In [4]:
# Urban population as a percentage of total population
show_weighted_average(years, 'SP.RUR.TOTL.ZS', 'SP.POP.TOTL', ['LDCs', 'Developing excluding LDCs'])

In [5]:
# Access to safely managed sanitation services
show_weighted_average(years, 'SH.STA.SMSS.ZS', 'SP.POP.TOTL', ['LDCs', 'Developing excluding LDCs'])

In [6]:
# Literacy rate, youth female (% of females ages 15-24)
years = list(map(str, list(range(2011,2018))))
show_weighted_average(years, 'SE.ADT.1524.LT.FE.ZS', 'SP.POP.TOTL', ['LDCs', 'Developing excluding LDCs'])

In [7]:
# Literacy rate, youth male (% of male ages 15-24)
show_weighted_average(years, 'SE.ADT.1524.LT.MA.ZS', 'SP.POP.TOTL', ['LDCs', 'Developing excluding LDCs'])

In [11]:
# School enrollment, tertiary
years = list(map(str, list(range(2010,2015))))
show_weighted_average(years, 'SE.TER.ENRR', 'SP.POP.TOTL', ['LDCs', 'Developing excluding LDCs'])

In [16]:
# Pupil teacher ration
years = list(map(str, list(range(2000,2018))))
show_weighted_average(years, 'SE.PRM.ENRL.TC.ZS', 'SP.POP.TOTL', ['LDCs', 'Developing excluding LDCs'])

In [17]:
# Population 0-14
show_weighted_average(years, 'SP.POP.0014.TO.ZS', 'SP.POP.TOTL', ['LDCs', 'Developing excluding LDCs', 'OECD'])

In [18]:
years = list(map(str, list(range(1960,2018))))
show_group_total(years, 'SP.POP.TOTL', ['LDCs'])

NameError: name 'show_group_total' is not defined

In [ ]:
data_group = get_data_table(years, 'SP.POP.TOTL', [['LDCs']])

In [ ]:
# Labor force
years = list(map(str, list(range(2000,2018))))
show_weighted_average(years, 'SL.EMP.VULN.FE.ZS', 'SL.TLF.TOTL.IN', ['LDCs', 'Developing excluding LDCs'])

In [ ]:
# Labor force
years = list(map(str, list(range(2000,2018))))
show_weighted_average(years, 'SH.STA.MMRT', 'SP.POP.TOTL.FE.IN', ['LDCs', 'Developing excluding LDCs'])